In [17]:
import pandas as pd
import json

In [18]:
def add_primary_data(matches_df, ranking_df, year, sports):
    # Add primary data - Creates the rows for the final export (Years, P1, P1 rank, P2, P2 rank, Winner, Match num, Total Matches)
    attribute_error_case = 0
    
    player1 = ""
    player2 = ""
    players_rank = []
    winner = ""
    
    player_position = {
        "volleyball": [1, 3, 6],
        "futsal": [0, 2, 4],
        "water_polo": [0, 2, 4],
        "roller_hockey": [0, 2, 4],
        "soccer": [1, 4, 5],
        "basketball": [0, 2, 4],
        "handball": [0, 2, 4],
        "ice_hockey": [0, 2, 4],
        "cricket": [0, 1, 2],
        "lacrosse": [0, 2, 4],
        "field_hockey": [0, 2, 4],
        "rugby": [0, 2, 4],
    }

    for i in range(len(matches_df)):
        # Finding players and winners 
        try: 
            #Player1 
            player1_rank = 0
            player1 = matches_df.iloc[i, player_position[sports][0]].strip() # This format helps since each column header is different 
            
            #Player 2 
            player2_rank = 0
            player2 = matches_df.iloc[i, player_position[sports][1]].strip() # Sport > Player 1/2/Winner column index 

            # Winner and Rank 
            winner = matches_df.iloc[i, player_position[sports][2]].strip()
            rank = list(ranking_df["Player"])

        except AttributeError: # Some player names are not string so strip() does not work. This escapes the cases
            attribute_error_case += 1
            continue 
        
        # Collecting the ranks for each player 
        if player1 in rank:
            player1_rank = rank.index(player1) + 1

        if player2 in rank:
            player2_rank = rank.index(player2) + 1
        else:
            continue

        players_rank.append([year, player1, player1_rank, player2, player2_rank, winner, i + 1, len(matches_df),])
    
    if attribute_error_case:
        print(f"Attribute Error cases for {sports} in {year} = {attribute_error_case}")
        
    return players_rank


In [19]:
def create_derived_fields(matches_rank_df):
    # Creating derived fields - Rank Diff, Progression, Match Diff, Levels
    matches_rank_df["Rank Diff"] = abs(matches_rank_df["Player 1 Rank"] - matches_rank_df["Player 2 Rank"])
    
    matches_rank_df["Progression"] = (matches_rank_df["Match Number"] / matches_rank_df["Total Matches"])
    
    matches_rank_df["Match Diff"] = abs(matches_rank_df["Total Matches"] - matches_rank_df["Match Number"])

    levels = []
    
    for difference in matches_rank_df["Match Diff"]:
        level = ""
        if difference == 0:
            level = "Finals"
        elif 1 <= difference <= 2:
            level = "Semifinals"
        elif 3 <= difference <= 6:
            level = "Quarterfinals"
        else:
            level = "Qualifications"

        levels.append(level)

    matches_rank_df["Tournament Level"] = levels

    return matches_rank_df

In [20]:
def drop_rows(matches_rank_df):
    # Drop rows - cleaning all the rows with "strange string"
    drop_index = []

    for i in range(len(matches_rank_df)):
        if matches_rank_df.iloc[i, 1] == 0:
            print(matches_rank_df.iloc[i, 1])
            drop_index.append(i)

    matches_rank_df = matches_rank_df.drop(drop_index)

    return matches_rank_df

In [21]:
def process(sport, year):
    matches_df = pd.read_excel("../data/" + sport + " Matches/" + sport + "_" + str(year) + ".xlsx")
    
    # Path too long
    if sport == 'roller_hockey' or sport == 'field_hockey':
        ranking_df = pd.read_excel("../data/" + sport + "R/" + sport + "_new_ranking_" + str(year) + ".xlsx")
    else:
        ranking_df = pd.read_excel("../data/" + sport + " New Ranking/" + sport + "_new_ranking_" + str(year) + ".xlsx")
    
    # Collecting all players primary data
    players_rank = add_primary_data(matches_df, ranking_df, year, sport)

    # Creating the df for export
    column = ["Year", "Player 1", "Player 1 Rank", "Player 2", "Player 2 Rank", "Winner", "Match Number", "Total Matches",]
    
    matches_rank_df = pd.DataFrame(players_rank, columns=column)

    matches_rank_df = create_derived_fields(matches_rank_df)
    matches_rank_df = drop_rows(matches_rank_df)

    return matches_rank_df

In [22]:
def main():
    # Importing json files for years and sports lists
    with open("years_reduced.json") as f:
        years_list = json.load(f)
    with open("sports.json") as g:
        sports_list = json.load(g)

    years_list['water_polo'] = years_list.pop('water_pollo')    
    sports_list['sports_list'][2] = 'water_polo'    
        
    sports = list(sports_list.values())
    
    for sport in sports[0]:
        year_list = years_list[sport] # unhash the json to years list 
        sport_df = pd.DataFrame()

        for year in year_list:            
            matches_rank_df = process(sport, year)
            frames = [sport_df, matches_rank_df]
            sport_df = pd.concat(frames, ignore_index=True)


        # final export for each sport given the sport_df has all the years appended 
        sport_df.to_excel("matches_rank_sports/matches_rank_" +sport+ ".xlsx", index=False)

In [23]:
# Importing json files for years and sports lists
with open("years_reduced.json") as f:
    years_list = json.load(f)
with open("sports.json") as g:
    sports_list = json.load(g)
    
years_list['water_polo'] = years_list.pop('water_pollo')    
sports_list['sports_list'][2] = 'water_polo'

sports = list(sports_list.values())

print(years_list,'\n')
print(sports_list,'\n')
print(sports,'\n')

{'volleyball': [1977, 1981, 1985, 1989, 1991, 1995, 1999, 2003, 2007, 2011, 2015, 2019], 'futsal': [1989, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020], 'roller_hockey': [1999, 2001, 2003, 2005, 2007, 2009, 2011, 2013, 2015], 'soccer': [1970, 1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014], 'basketball': [1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016, 2020], 'handball': [1976, 1980, 1984, 1988, 1992, 1996, 2000, 2008, 2012, 2016, 2020], 'ice_hockey': [1972, 1976, 1980, 1984, 1988, 1992, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2022], 'cricket': [1975, 1979, 1983, 1987, 1992, 1996, 1999, 2003, 2007, 2011, 2015, 2019], 'lacrosse': [1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014], 'field_hockey': [1971, 1973, 1975, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 2010, 2014, 2018, 2023], 'rugby': [2003, 2007, 2011, 2015, 2019, 1999, 1995, 1991, 1987], 'water_polo': [1979, 1981, 1983, 1985, 1993, 1995, 1999, 2002, 2006, 2010, 2

In [24]:
main()